## Enriquecimiento del dataset
En este proceso se realizan 2 principales acciones: 1) Seleccionar preguntas según los indicadores elegidos. 2) Ponderar valores de las preguntas elegidas.

## Indicadores
Se eligieron los siguientes indicadores para darle un valor agregado al dataset generado. Los mismos están enfocados desde la perspectiva de reflejar las opiniones de los encuestados; esto siguiendo siempre un enfoque de apoyo o beneficio a la educación.  
* **Utilidad:** La utilidad se refiere a la capacidad de las TICs para satisfacer las necesidades y expectativas de los encuestados. Se puede preguntar sobre la calidad de los servicios y herramientas disponibles, la efectividad de las TICs en la realización de tareas, la satisfacción con los servicios y la capacidad de las TICs para mejorar la vida cotidiana.
* **Accesibilidad:** La accesibilidad se refiere a la facilidad con la que los encuestados pueden acceder a las TICs. Se puede preguntar sobre el acceso a la tecnología, la conexión a Internet, la frecuencia de uso y la facilidad de uso.


In [46]:
import pandas as pd
import numpy as np
pd.options.display.max_colwidth = 100

In [12]:
df = pd.read_csv("enal_final_clnd.csv", index_col=0)
df.head()

,P1,P2,P1A,P1B,P1C,P1D,P6,P11A,P11B,P11C,...,P63A,P63B,P68,P131A,P131B,P131C,P148,P149,P152,P160
0,18,1,NaN,NaN,NaN,NaN,1,4.0,5.0,NaN,...,1,2.0,1,5,NaN,NaN,1,3,6,La Paz
4,16,1,NaN,NaN,NaN,NaN,2,5.0,NaN,NaN,...,1,2.0,1,1,2.0,NaN,2,2,6,La Paz
9,18,1,NaN,NaN,NaN,NaN,1,2.0,3.0,4.0,...,1,2.0,2,5,NaN,NaN,1,3,6,La Paz
11,20,1,NaN,NaN,NaN,NaN,1,2.0,3.0,4.0,...,2,NaN,2,6,NaN,NaN,1,6,6,La Paz
17,18,1,NaN,NaN,NaN,NaN,2,4.0,5.0,NaN,...,2,NaN,1,1,2.0,6.0,2,3,6,La Paz


## Ponderaciones y consideraciones
Para determinada serie de preguntas se seleccionan ciertas ponderaciones que se describen a continuación en formato de un objeto JSON.
Se asignan 0-1 para respuestas del tipo Sí-No(1-2) y para las de selección múltiple se ponderan las respuestas de acuerdo a la opción que corresponde.


In [28]:
# Ponderaciones para preguntas del indicador: Utilidad.
# para preguntas ^p11
op1_util = {
    1: 0.8,#"Herramienta de trabajo"
    2: 0.8,#"Escuchar música o ver videos",
    3: 0.5,#"Juegos",
    4: 1,#"Conectarse a Internet",
    5: 1,#"Estudio",
}
max_pu1 = sum(op1_util.values()) # Máximo puntaje posible

# para ^p46 y ^p27
op2_util = {
    1: 0.5,#"Contactar amigos o familiares",
    2: 1,#"Buscar información",
    3: 0,#"Comprar, vender o permutar",
    4: 1,#"Fines académicos",
    5: 0.8,#"Trabajo",
    6: 0.8,#"Ver noticias",
    7: 0.8,#"Descargar archivos",
    8: 0,#"Jugar",
    9: 0.5,#"Escuchar música o ver videos",
    10: 0,#"Redes sociales",
    11: 0.8,#"Correo electrónico",
    12: 0,#"Otro"
 }
max_pu2 = sum(op2_util.values()) # Máximo puntaje posible

# para: P68
op3_util = {
            1: 1,#"El Colegio",
            2: 1,#"La Universidad",
            3: 0,#"No usa/no usó"
}
# max_pu1 = sum(pu1.values()) # Máximo puntaje posible

indicators = ('accesibilidad', 'utilidad')

### Preguntas seleccionadas según los indicadores
Estas preguntas fueron seleccionadas según los indicadores previos y se encuentran en el archivo de **final_questions.txt**. Dicho archivo contiene las preguntas seleccionadas divididas por indicadores que las representan y de la forma:  
**código de pregunta: pregunta**

In [14]:
# Recuperamos las preguntas seleccionadas
indicators_qs = dict.fromkeys(indicators, [])
indicator = ""
text = ""
aq = []
with open("final_questions.txt", "r", encoding="utf-8") as f:
    for qc in f:
        q = qc.split(":")
        if q[0] in indicators:
            if aq:
                indicators_qs[indicator] = aq
                aq = []
            indicator = q[0].strip()
        elif len(q) == 2 and q[0] not in indicators:
            aq.append(q[0])
    indicators_qs[indicator] = aq

In [15]:
indicators_qs

{'accesibilidad': ['P2', 'P6', 'P12', 'P16', 'P17', 'P20', 'P23', 'P68'],
 'utilidad': ['P11A',
  'P11B',
  'P11C',
  'P11F',
  'P27A',
  'P27B',
  'P27C',
  'P46A',
  'P46B',
  'P46C',
  'P46E',
  'P46F',
  'P46G',
  'P46H',
  'P46I']}

In [16]:
# Columnas seleccionadas de accesibilidad
acc_df = df[indicators_qs["accesibilidad"]]
nq_acc = len(indicators_qs["accesibilidad"])
acc_df.head()

,P2,P6,P12,P16,P17,P20,P23,P68
0,1,1,1,1,2,1,1,1
4,1,2,1,2,1,1,3,1
9,1,1,1,1,2,1,3,2
11,1,1,1,1,2,2,2,2
17,1,2,1,2,2,1,3,1


In [22]:
# Evaluando las preguntas del indicador accesibilidad
acces_col = []
for _, row in acc_df.iterrows():
    sum_row = 0
    for col, val in row.items():
        if val == 1 or (col in ("P23", "P68") and val != 3): # No sumamos la respuesta: 3 Ninguno
            sum_row += 1
    acces_col.append(sum_row/nq_acc)

In [25]:
# Columnas seleccionadas de utilidad
util_df = df[indicators_qs["utilidad"]]
nq_util = len(indicators_qs["utilidad"])
util_df.head()

,P11A,P11B,P11C,P11F,P27A,P27B,P27C,P46A,P46B,P46C,P46E,P46F,P46G,P46H,P46I
0,4.0,5.0,NaN,NaN,2.0,5.0,7.0,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5,2.0,NaN,NaN,NaN,NaN,NaN,NaN
9,2.0,3.0,4.0,5.0,NaN,NaN,NaN,8,9.0,10.0,NaN,NaN,NaN,NaN,NaN
11,2.0,3.0,4.0,5.0,NaN,NaN,NaN,1,2.0,4.0,NaN,NaN,NaN,NaN,NaN
17,4.0,5.0,NaN,NaN,NaN,NaN,NaN,1,7.0,10.0,11.0,NaN,NaN,NaN,NaN


In [48]:
# Procesando preguntas del factor de utilidad
util_col1 = []
for _, row in util_df.iterrows():
    sum_row = 0
    for col, val in row.items():
        if col.startswith("P11"):
            try:
                sum_row += op1_util[int(val)]
            except:
                pass
    util_col1.append(round(sum_row/max_pu1, 3))

util_col2 = []
for _, row in util_df.iterrows():
    sum_row = 0
    for col, val in row.items():
         if col.startswith(("P27", "P46")):
            try:
                sum_row += op2_util[int(val)]
            except:
                pass
    util_col2.append(round(sum_row/max_pu2, 3))

util_col = [round(i, 3) for i in np.add(util_col1, util_col2)/2]

## Finalmente incluimo todas las columnas calculadas que pertenecen los factores elegidos

In [53]:
df["accesibilidad"] = acces_col
df["utilidad"] = util_col

In [54]:
df.head()

,P1,P2,P1A,P1B,P1C,P1D,P6,P11A,P11B,P11C,...,P68,P131A,P131B,P131C,P148,P149,P152,P160,accesibilidad,utilidad
0,18,1,NaN,NaN,NaN,NaN,1,4.0,5.0,NaN,...,1,5,NaN,NaN,1,3,6,La Paz,0.875,0.574
4,16,1,NaN,NaN,NaN,NaN,2,5.0,NaN,NaN,...,1,1,2.0,NaN,2,2,6,La Paz,0.625,0.267
9,18,1,NaN,NaN,NaN,NaN,1,2.0,3.0,4.0,...,2,5,NaN,NaN,1,3,6,La Paz,0.750,0.443
11,20,1,NaN,NaN,NaN,NaN,1,2.0,3.0,4.0,...,2,6,NaN,NaN,1,6,6,La Paz,0.750,0.604
17,18,1,NaN,NaN,NaN,NaN,2,4.0,5.0,NaN,...,1,1,2.0,6.0,2,3,6,La Paz,0.500,0.414


In [55]:
# Exportamos el nuevo dataset enriquecido
df.to_csv("enrich_enal_final_clnd.csv")